In [1]:
!pip install transformers

In [2]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [20]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

root_dir = "/gdrive/My Drive/nlpbook/1_project"
import sys
sys.path.append(root_dir)

Mounted at /gdrive


In [5]:
# GPU 사용
device = torch.device("cuda")

# Dataset 만들어서 불러오기

In [36]:
class NSMCDataset(Dataset):

  def __init__(self, dataset):
    self.dataset = dataset


    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.iloc[idx].values
    if len(row)==2:
      text = row[0]
      y = row[1]
    else :
      text=row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    if len(row)==2:
      return input_ids, attention_mask, y
    else :
      return input_ids, attention_mask, -1

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split



# 결과 확인

path = '/gdrive/My Drive/nlpbook/1_project/comments_labeled.csv'
df = pd.read_csv(path).drop(['date','recomm','unrecomm'],axis=1)

train_df, test_df = train_test_split(df, train_size=0.8, random_state=42)

train_dataset = NSMCDataset(train_df)
test_dataset = NSMCDataset(test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

            labeled
count  15600.000000
mean       0.453654
std        0.497863
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
           labeled
count  3900.000000
mean      0.466154
std       0.498917
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000


In [9]:
train_dataset.__getitem__(1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(tensor([    2,   878, 29983, 30162,  4312,  3084, 28769, 10431,    20, 30124,
          4668, 30913,   168,     1,    20,   260,   260,   260,   260,   148,
          1027,   484, 30076,     1,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:

train_dataset = NSMCDataset(train_df)
test_dataset = NSMCDataset(test_df)

                 id          label
count  1.461820e+05  146182.000000
mean   6.779186e+06       0.498283
std    2.919223e+06       0.499999
min    3.300000e+01       0.000000
25%    4.814832e+06       0.000000
50%    7.581160e+06       0.000000
75%    9.274760e+06       1.000000
max    1.027815e+07       1.000000
                 id         label
count  4.915700e+04  49157.000000
mean   6.752945e+06      0.502695
std    2.937158e+06      0.499998
min    6.010000e+02      0.000000
25%    4.777143e+06      0.000000
50%    7.565415e+06      1.000000
75%    9.260204e+06      1.000000
max    1.027809e+07      1.000000


# Create Model

In [10]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

# Learn

In [13]:
epochs = 5
batch_size = 16

In [14]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/975 [00:00<?, ?it/s]

Batch Loss: 68.60712593793869 Accuracy: tensor(0.5544, device='cuda:0')
Batch Loss: 134.4366130232811 Accuracy: tensor(0.5938, device='cuda:0')
Batch Loss: 193.21394848823547 Accuracy: tensor(0.6290, device='cuda:0')
Batch Loss: 246.26301822066307 Accuracy: tensor(0.6566, device='cuda:0')
Batch Loss: 295.1801068931818 Accuracy: tensor(0.6809, device='cuda:0')
Batch Loss: 343.88947108387947 Accuracy: tensor(0.6956, device='cuda:0')
Batch Loss: 390.60509791970253 Accuracy: tensor(0.7069, device='cuda:0')
Batch Loss: 433.86077477037907 Accuracy: tensor(0.7193, device='cuda:0')
Batch Loss: 477.92853307724 Accuracy: tensor(0.7279, device='cuda:0')
Train Loss: 510.63994267582893 Accuracy: tensor(0.7335, device='cuda:0')


  0%|          | 0/975 [00:00<?, ?it/s]

Batch Loss: 38.37289088964462 Accuracy: tensor(0.8294, device='cuda:0')
Batch Loss: 77.4960667192936 Accuracy: tensor(0.8294, device='cuda:0')
Batch Loss: 115.6597246080637 Accuracy: tensor(0.8290, device='cuda:0')
Batch Loss: 152.53852029889822 Accuracy: tensor(0.8292, device='cuda:0')
Batch Loss: 191.1901367381215 Accuracy: tensor(0.8290, device='cuda:0')
Batch Loss: 223.835489846766 Accuracy: tensor(0.8339, device='cuda:0')
Batch Loss: 256.6214036270976 Accuracy: tensor(0.8377, device='cuda:0')
Batch Loss: 291.28368339687586 Accuracy: tensor(0.8402, device='cuda:0')
Batch Loss: 325.1210638433695 Accuracy: tensor(0.8425, device='cuda:0')
Train Loss: 350.7617199495435 Accuracy: tensor(0.8426, device='cuda:0')


  0%|          | 0/975 [00:00<?, ?it/s]

Batch Loss: 34.03486938774586 Accuracy: tensor(0.8581, device='cuda:0')
Batch Loss: 64.81550778448582 Accuracy: tensor(0.8656, device='cuda:0')
Batch Loss: 96.33416009694338 Accuracy: tensor(0.8617, device='cuda:0')
Batch Loss: 125.82006019353867 Accuracy: tensor(0.8667, device='cuda:0')
Batch Loss: 157.53642119467258 Accuracy: tensor(0.8678, device='cuda:0')
Batch Loss: 186.18810473382473 Accuracy: tensor(0.8706, device='cuda:0')
Batch Loss: 217.32362033426762 Accuracy: tensor(0.8710, device='cuda:0')
Batch Loss: 247.59734281152487 Accuracy: tensor(0.8712, device='cuda:0')
Batch Loss: 276.02222921699286 Accuracy: tensor(0.8725, device='cuda:0')
Train Loss: 299.7590943425894 Accuracy: tensor(0.8721, device='cuda:0')


  0%|          | 0/975 [00:00<?, ?it/s]

Batch Loss: 27.16669524461031 Accuracy: tensor(0.8887, device='cuda:0')
Batch Loss: 55.391755506396294 Accuracy: tensor(0.8872, device='cuda:0')
Batch Loss: 85.15069396421313 Accuracy: tensor(0.8846, device='cuda:0')
Batch Loss: 112.43220066651702 Accuracy: tensor(0.8848, device='cuda:0')
Batch Loss: 140.47170753777027 Accuracy: tensor(0.8866, device='cuda:0')
Batch Loss: 166.023382473737 Accuracy: tensor(0.8890, device='cuda:0')
Batch Loss: 193.71680847927928 Accuracy: tensor(0.8891, device='cuda:0')
Batch Loss: 220.6475730650127 Accuracy: tensor(0.8895, device='cuda:0')
Batch Loss: 249.314924813807 Accuracy: tensor(0.8892, device='cuda:0')
Train Loss: 268.94207989424467 Accuracy: tensor(0.8892, device='cuda:0')


  0%|          | 0/975 [00:00<?, ?it/s]

Batch Loss: 21.784815706312656 Accuracy: tensor(0.9131, device='cuda:0')
Batch Loss: 46.38739180192351 Accuracy: tensor(0.9056, device='cuda:0')
Batch Loss: 70.30111301317811 Accuracy: tensor(0.9073, device='cuda:0')
Batch Loss: 94.26110593415797 Accuracy: tensor(0.9077, device='cuda:0')
Batch Loss: 117.63134946860373 Accuracy: tensor(0.9093, device='cuda:0')
Batch Loss: 143.19962317310274 Accuracy: tensor(0.9067, device='cuda:0')
Batch Loss: 166.29073023609817 Accuracy: tensor(0.9073, device='cuda:0')
Batch Loss: 191.13424297235906 Accuracy: tensor(0.9072, device='cuda:0')
Batch Loss: 217.6335818450898 Accuracy: tensor(0.9052, device='cuda:0')
Train Loss: 236.00482449494302 Accuracy: tensor(0.9049, device='cuda:0')


In [16]:
losses, accuracies

([510.63994267582893,
  350.7617199495435,
  299.7590943425894,
  268.94207989424467,
  236.00482449494302],
 [tensor(0.7335, device='cuda:0'),
  tensor(0.8426, device='cuda:0'),
  tensor(0.8721, device='cuda:0'),
  tensor(0.8892, device='cuda:0'),
  tensor(0.9049, device='cuda:0')])

# 테스트 데이터셋 정확도 확인하기

In [31]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/244 [00:00<?, ?it/s]

Accuracy: tensor(0.8618, device='cuda:0')


# 훈련에 사용되지않은 댓글 천개 인덱싱해서 가져와 테스트

In [19]:
# 모델 저장하기

model_save_path = '/gdrive/My Drive/nlpbook/1_project/model.pt'

torch.save(model.state_dict(), model_save_path)

In [44]:
test_df = pd.read_csv('/gdrive/My Drive/nlpbook/1_project/movie_comments(51090).csv').drop(['date','recomm','unrecomm'],axis=1)[30000:31000]
test_df

,comment
30000,시리즈온 한국어자막 나오게 하는 방법좀 알려주세요. ㅜㅜㅜㅜㅜ
30001,"나왔구나, 마침내"
30002,4-5G말고 15G 정도 파일로 업로드 해주면 좋겠다.
30003,블루레이 출시는 안 하면서 질낮은 파일을 비싸게 파는 이유는 뭘까?
30004,왜 맥os는... 지원하지 않는 건가요. 내 마음: 무너지고 깨어짐
...,...
30995,아 슬프네요 ㅠㅠ
30996,tmfvmekd
30997,너무 비쌉니다. 1200원으로 해 주세요.
30998,강추


In [48]:

model = ElectraForSequenceClassification.from_pretrained("/gdrive/My Drive/nlpbook/1_project/model.pt")
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict(text):
    inputs = tokenizer(text,
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    return predicted_class

test_df['predicted_class'] = test_df.iloc[:, 0].apply(predict)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [49]:
test_df[['comment', 'predicted_class']]

,comment,predicted_class
30000,시리즈온 한국어자막 나오게 하는 방법좀 알려주세요. ㅜㅜㅜㅜㅜ,0
30001,"나왔구나, 마침내",0
30002,4-5G말고 15G 정도 파일로 업로드 해주면 좋겠다.,0
30003,블루레이 출시는 안 하면서 질낮은 파일을 비싸게 파는 이유는 뭘까?,0
30004,왜 맥os는... 지원하지 않는 건가요. 내 마음: 무너지고 깨어짐,0
...,...,...
30995,아 슬프네요 ㅠㅠ,1
30996,tmfvmekd,0
30997,너무 비쌉니다. 1200원으로 해 주세요.,0
30998,강추,1
